In [2]:
from IPython.core.display import HTML

HTML("""
<style>
.rendered_html {
    font-family: Monaco, monospace;
    font-size: 12px;
}
</style>
""")

#### LEARNING PARTICLE DYNAMICS FOR MANIPULATING RIGID BODIES, DEFORMABLE OBJECTS, AND FLUIDS

**DPI-Net** stands for Dynamic Particle Interactioin Network

based on the [paper](http://dpi.csail.mit.edu/dpi-paper.pdf)

here we utilise 3D particles as the
scene representation for the environment and develop dynamic particle interaction networks (DPI-Nets) for learning particle dynamics, focusing on capturing the dynamic, hierarchical, and long-range interactions of particles

<img src=./images/dpiNet.png width=600 style="display: block; margin: auto;">

DPI-Nets combine three key features for eﬀective particle-based
simulation and control: *multi-step spatial propagation, hierarchical
particle structure, and dynamic interaction graphs.*

Learning-Based 3D Particle Dynamics

Preliminaries

The learning-based 3D particle dynamics model is developed based on interaction networks (INs). We first outline how INs represent the physical system and then extend them to accommodate particle-based dynamics.
We describe the scene representation that represents the interactions within a physical system as a directed graph, $z = (V, E)$, where vertices $V = {v_i}$ correspond to objects and edges $E = {e_k}$ denote relations. Specifically, $v_i = (x_i, r_i^v)$, where $x_i = (q_i, \dot{q}_i)$ signifies the state of object $i$, encompassing its position $q_i$ and velocity $\dot{q}_i$. The term $r_i^v$ designates its attributes (e.g., mass and radius). For the relation, we define $e_k = (a_k, b_k, r_k^e), 1 \leq a_k, b_k \leq |V|$, where $a_k$ is the receiver, $b_k$ is the sender, and both are integers. $r_k^e$ represents the type and attributes of relation $k$ (e.g., collision and spring connection).
As discussed in Section 2.1, the objective is to construct a learnable physical engine capable of capturing the underlying physical interactions using function approximators $f_{\theta}$. The learned model can subsequently be used for system identification and predicting the future state from the current interaction graph as $z_{t+1} = f_{\theta}(z_t, u_t)$, where $z_t$ in this section denotes the particle-based scene representation at time $t$.
Interaction networks: Battaglia et al. propose INs, a general-purpose, learnable physics engine that performs object- and relation-centric reasoning about physics. INs define an object function $f_V$ and a relation function $f_E$ to model objects and their relations in a compositional way. The future state at time $t + 1$ is predicted as
$$e_{k,t} = f_E(v_{a_k,t}, v_{b_k,t}, r_k^e) \quad k = 1, \ldots, |E|,$$
$$\hat{v}{i,t+1} = f_V \left( v{i,t}, \sum_{k \in \mathcal{N}i} \hat{e}{k,t} \right) \quad i = 1, \ldots, |V|,$$
where $v_{i,t} = (x_{i,t}, r_i^v)$ denotes object $i$ at time $t$, $a_k$ and $b_k$ are the receiver and sender of relation $e_k$, respectively, and $\mathcal{N}_i$ denotes the relations where object $i$ is the receiver.


Propagation Networks
A limitation of INs is that at every time step $t$, it only considers local information in the graph $z_t = (V_t, E_t)$ and cannot handle instantaneous changes in forces/accelerations in other bodies, which however is a common phenomenon in rigid-body dynamics. Li et al. proposed propagation networks to approximate the instantaneous force changes by doing multi-step message passing. Specifically, they first employed the ideas on fast training of RNNs to encode the shared information beforehand and reuse them along the propagation steps. The encoders for objects are denoted as $f_V^{\text{enc}}$ and the encoder for relations as $f_E^{\text{enc}}$, where we denote
$$c_{k,t}^e = f_E^{\text{enc}}(v_{a_k,t}, v_{b_k,t}, r_k^e) \quad k = 1, \dots, |E|,$$
$$c_{i,t}^v = f_V^{\text{enc}}(v_{i,t}), \quad i = 1, \dots, |V|.$$
At time $t$, denote the propagating influence from relation $k$ at propagation step $l$ as $\hat{e}{k,t}^l$ and the propagating influence from object $i$ as $\hat{v}{i,t}^l$. For step $1 \leq l \leq L$, propagation can be described as
Step 0:
$$\hat{v}_{i,t}^0 = 0, \quad i = 1, \dots, |V|,$$
Step $l = 1, \dots, L$:
$$\hat{e}_{k,t}^l = f_E^e \left( \hat{e}_{k,t}^{l-1}, \hat{v}_{a_k,t}^{l-1} \right), \quad k = 1, \dots, |E|,$$
$$\hat{v}_{i,t}^l = f_V \left( \hat{v}_{i,t}^{l-1}, \sum_{k \in N_i} \hat{e}_{k,t}^l \hat{v}_{i,t}^{l-1} \right), \quad i = 1, \dots, |V|,$$
Output:
$$\hat{v}_{i,t+1} = f_V^{\text{output}}(\hat{v}_{i,t}^L), \quad i = 1, \dots, |V|,$$
where $f_V$ denotes the object propagator and $f_E$ denotes the relation propagator.


#### Hierarchical modelling -> for long range dependence

Model predictive control using shooting methods -> Using SGD between the predicted $z_p$ by applying the control input $u_{1:H}$ (here $H$ is time horizon) and the goal $z_g$

### Control:

so far, We leverage **dynamic particle interaction networks** for control tasks
in both simulation and the real world. Because trajectory optimisation using the shooting method can easily be stuck to a local minimum, we first randomly sample $N_{sample}$ control sequences, similar
to the [**MPPI**](https://arxiv.org/pdf/1509.01149) algorithm, and select the
best-performing one according to the rollouts of our learned model.
We then optimise it via the shooting method using our model’s gradients. We also use **online system identification** to further improve the model’s performance.

#### Notes on MPPI worth knowing:

MPPI (Model Predictive Path Integral) Control is a sampling-based, derivative-free control algorithm for nonlinear, stochastic systems. It solves the stochastic optimal control problem by sampling trajectories, evaluating them under a cost function, and then updating control actions accordingly.

It’s based on path integral control theory, which reformulates the stochastic optimal control problem into an expectation under a modified distribution. MPPI applies this iteratively in a **model predictive control (MPC)** framework—recomputing the optimal control at each timestep using a *rolling horizon*.

----
from the paper itself -> [paper](http://dpi.csail.mit.edu/dpi-paper.pdf)

DPI-Nets combine three key features for effective particle-based simulation and control: multi-step
spatial propagation, hierarchical particle structure, and dynamic interaction graphs. In particular,
it employs dynamic interaction graphs, built on the fly throughout manipulation, to capture the
meaningful interactions among particles of deformable objects and fluids. The use of dynamic graphs
allows neural models to focus on learning meaningful interactions among particles, and is crucial for
obtaining good simulation accuracy and high success rates in manipulation. As objects deform when
robots interact with them, a fixed interaction graph over particles is insufficient for robot manipulating
non-rigid objects.

An inspiring concurrent work from Mrowca et al. (2018) explored learning to approximate
particle dynamics of deformable shapes with the **Hierarchical Relation Network (HRN)**. Compared
with these papers, we introduce state-specific modeling and dynamic graphs for accurate forward
prediction for different states of matter (rigid bodies, deformable shapes, fluids). We also demonstrate
how the **learned dynamics model**(based on MPPI -> $N_{sample}$ and roll-out and optim) can be used for control in both simulation and real world.

The goal is to build a learnable physical engine to capture the underlying physical interactions using
function approximators $\phi$. The learned model can then be used *to infer the system dynamics* and
predict the future from the current interaction graph as $G_{t+1} = \phi(G_t)$, where $G_t$ denotes the scene
state at time $t$.

**Interaction Network**:

interaction networks (IN), a general-
purpose, **learnable physics engine** that performs object- and relation-centric reasoning about
physics. INs define an object function $f_O$ and a relation function $f_R$ to model objects and
their relations in a compositional way.

#### PropNet:

$?$ In the PropNet section, why do we encode the shared information (like relation info), how is it used, and why is it necessary?

### DYNAMIC PARTICLE INTERACTION NETWORKS

How is the graph built dynamically?

For each time step:
1.	Particles = Nodes.
2.	Edges are formed by:
    - Proximity: If particle i is within a cutoff radius of particle j, create an edge.
    - Type constraints: Some edges only form if the particles are part of a fluid or deformable object.
    - Predefined relations: For rigid objects or articulated structures, edge types may be fixed and reused.

In [ ]:
# sample dynamic graph construction for a particle system
# section 3.2.1 of the paper 
import torch
from torch_scatter import scatter
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing

def build_dynamic_graph(positions, cutoff_radius, relation_types=None):
    """
    Build a graph dynamically based on particle positions.
    
    Args:
        positions (Tensor): shape (N, 3) for N particles in 3D.
        cutoff_radius (float): neighborhood radius for edge formation.
        relation_types (optional): additional metadata to attach to edges.

    Returns:
        edge_index (Tensor): shape (2, E), edge list.
        edge_attr (Tensor): shape (E, F), relation features if applicable.
    """
    N = positions.size(0)
    
    # Compute pairwise distances (inefficient, for illustration)
    diff = positions.unsqueeze(1) - positions.unsqueeze(0)  # (N, N, 3)
    dist = torch.norm(diff, dim=2)  # (N, N)

    # Threshold to find neighbors
    edge_mask = (dist < cutoff_radius) & (dist > 0)  # remove self-edges

    edge_index = edge_mask.nonzero(as_tuple=False).T  # shape (2, E)

    # Optional: compute relation features (like distance vector or type)
    edge_attr = diff[edge_mask]  # (E, 3), could add type or other info here

    return edge_index, edge_attr

#### Hierarchical modeling for long-range dependence. 

in PropNet, we need a large $L$ to account for long-range dependence, so we use Hierarchical modeling for this object that need long-range dependence.

We employ a multi-stage propagation paradigm (Root and Leaf concepts)

this hierarchical modeling is and interaction graph based on hierarchical modeling and propagation rules are defined on different types of objects as follows:
- Rigid bodies:All the particles in a rigid body are globally coupled; hence for each rigid
object, we define a hierarchical model to propagate the effects. After the multi-stage
propagation, we average the signals on the particles to predict a rigid transformation (rotation
and translation) for the object. The motion of each particle is calculated accordingly. For
each particle, we also include its *offset to the center-of-mass* to help determine the torque.

- Elastic/Plastic objects: For elastically deforming particles, only using the current position
and velocity as the state is not sufficient, as it is not clear where the particle will be
restored after the deformation. Hence, we include the particle state with the resting position
to indicate the place where the particle should be restored. When coupled with plastic
deformation, the resting position might change during an interaction. Thus, we also infer the
motion of the resting position as a part of the state prediction. We use hierarchical modeling
for this category but predict next state for each particles individually.

- Fluids: For fluid simulation, one has to enforce density and incompressibility, which can be
effectively achieved by only considering a small neighborhood for each particle (Macklin &
M¨ uller, 2013). Therefore, we do not need hierarchical modeling for fluids. We build edges
dynamically, connecting a fluid particle to its neighboring particles. The strong *inductive bias* 
leveraged in the fluid particles allows good performance even when tested on data
outside training distributions.

For the interaction between different materials, two directed edges are generated for any pairs of
particles that are closer than a certain distance.

#### Control on the Learned Dynamics

Model-based methods offer many advantages when comparing with their model-free counterparts,
such as generalization and sample efficiency. However, for cases where an accurate model is hard
to specify or computationally prohibitive, a data-driven approach that learns to approximate the
underlying dynamics becomes useful.

Function approximators such as neural networks are naturally differentiable. We can rollout using the
learned dynamics and optimize the control inputs by minimizing a loss between the simulated results
and a target configuration. In cases where certain physical parameters are unknown, we can perform
online system identification by minimizing the difference between the model’s prediction and the
reality. [code](../src/DPI-Particle-based/concept.py)

<img src=./images/control_on_learned_dynamics.png width=500 style="display: block; margin: auto;">

End of the paper experimentation

----